In [5]:
import pandas as pd
import numpy as np
import pickle

import io
import os
import sys
import json
import urllib3
import multiprocessing

from PIL import Image
from tqdm import tqdm
from urllib3.util import Retry

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [6]:
## parsing the data and making a list of tuples of file path and image url ##

def parse_dataset(data, outdir):
    
    fnames_urls = []
    for index, row in data.iterrows():
        url = row.imageUrlStr.split(';')[0]
        fname = os.path.join(outdir, "{}.jpg".format(row.productId))
        fnames_urls.append((fname, url))
    
    return fnames_urls

In [7]:
## Download the Images ##

def download_image(fnames_and_urls):
    
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        try:
            http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
            response = http.request("GET", url)
            image = Image.open(io.BytesIO(response.data))
            image_rgb = image.convert("RGB")
            imResize = image_rgb.resize((224,224), Image.ANTIALIAS)
            imResize.save(fname, format='JPEG', quality=90)
        except:
            pass

In [9]:
## Loading the download pickle file and calling the download function using multiprocesssing ##
## Downloading the images was taking a lot of time and was unable to run more than 4 processes ##
## So ran the same script on a cloud instance for downloading all the images of tops ##

if __name__ == '__main__':
    
    with open('download_df.pickle', 'rb') as handle:
        download_df = pickle.load(handle)
    outdir = "/home/ankan/Projects/infilect/images"
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    fnames_urls = parse_dataset(download_df, outdir)
    
    pool = multiprocessing.Pool(processes=4)
    with tqdm(total=len(fnames_urls)) as progress_bar:
        for _ in pool.imap_unordered(download_image, fnames_urls):
            progress_bar.update(1)

  6%|▌         | 20118/350638 [2:35:02<131:51:10,  1.44s/it]


KeyboardInterrupt: 